# Merge facility information

Merge facility data from HCRIS (Healthcare Cost Reporting Information System) and DH (Definitive Healthcare) datasets.

In [1]:
import os
import pandas as pd
import geopandas as gpd
import numpy as np
from os.path import join, isdir

from covidcaremap.geo import spatial_join_facilities
from covidcaremap.data import processed_data_path, external_data_path
from covidcaremap.mapping import HospMap
from covidcaremap.merge import Matcher

Geocoded datasets

In [2]:
hcris = gpd.read_file(processed_data_path('usa_facilities_hcris_geocoded.geojson'), encoding='utf-8')
dh = gpd.read_file(processed_data_path('dh_geocoded_v1_0326202.geojson'), encoding='utf-8')

Rename columns to match between datasets

In [3]:
hcris.rename(columns={
    'ST_ABBR': 'STATE_NAME',
    'Zip_Code': 'ZIP_CODE'
}, inplace=True)

dh['STATE_NAME'] = dh['ST_ABBR']

List of states to iterate over

In [4]:
all_states = hcris['STATE_NAME'].unique()

Collect data set, data set name, and data set uid for matching

In [5]:
hcris_info = (hcris, 'HCRIS', 'Provider Number')
dh_info = (dh, 'DH', 'OBJECTID')

In [6]:
def match_by_state(d1, d2, map_dir=None, str_match_method='name'):
    d1, name1, uid1 = d1
    d2, name2, uid2 = d2
    
    if not os.path.isdir(processed_data_path(map_dir)):
        os.mkdir(processed_data_path(map_dir))
    
    state_matches = {}
    for state in all_states:
        print('Matching facilities in {}'.format(state))
        if state not in state_matches.keys():
            d1_s = d1[d1['STATE_NAME'] == state].reset_index().copy()
            d2_s = d2[d2['STATE_NAME'] == state].reset_index().copy()
            m = Matcher(d1_s, d2_s, uid1, uid2)
            m.match_point_set((100, 500), 10, str_match_method=str_match_method)
            if map_dir:
                all_map = m.map_all((name1, name2), ['match source', 'dist_apart'])
                all_map.add_layer_selector()
                all_map.save(join(processed_data_path('{}'.format(map_dir)), '{}.html'.format(state)))
            state_matches[state] = m
    
    ds = {
        f'{name1}_matched': [],
        f'{name2}_matched': [],
        f'{name1}_unmatched': [],
        f'{name2}_unmatched': [],
        'matching_dfs': []
    }
    
    for _, v in state_matches.items():
        ds[f'{name1}_matched'].append(v.d1_matched)
        ds[f'{name2}_matched'].append(v.d2_matched)
        ds[f'{name1}_unmatched'].append(v.d1_unmatched)
        ds[f'{name2}_unmatched'].append(v.d2_unmatched)
        ds['matching_dfs'].append(v.matching_key_df())
    
    for k, v in ds.items():
        ds[k] = pd.concat(v)
        if isinstance(ds[k], gpd.GeoDataFrame):
            ds[k] = ds[k].to_crs('epsg:4326')
    
    print('------------')
    n_matched = len(ds['matching_dfs'])
    n_unmatched = len(ds[f'{name1}_unmatched'])
    n_total = n_matched + n_unmatched
    pct_matched = round((n_matched / n_total) * 100, 1)
    print(f'{name1} to {name2} matches: {pct_matched}% ({n_matched} of {n_total})')
    
    return state_matches, ds

Match the two data sets state by state, writing off folium maps

In [7]:
hcris_to_dh_matches, hcris_to_dh_data = match_by_state(
    hcris_info, 
    dh_info, 
    'state_validation_maps_03-31-21_hcris-to-dh')

HtoD_matches = hcris_to_dh_data['matching_dfs'].astype(str)
HtoD_matches.to_csv(processed_data_path('HCRIS_to_DH_matching_key.csv'), index=False)

Matching facilities in AL
Completed matching and deduping facilities, matched 109 of 136
Matching facilities in AK
Completed matching and deduping facilities, matched 23 of 26
Matching facilities in AZ
Completed matching and deduping facilities, matched 104 of 125
Matching facilities in AR
Completed matching and deduping facilities, matched 95 of 109
Matching facilities in CA
Completed [250] of 445 facilities, prelim matched 230
Completed matching and deduping facilities, matched 401 of 445
Matching facilities in CO
Completed matching and deduping facilities, matched 101 of 110
Matching facilities in CT
Completed matching and deduping facilities, matched 40 of 43
Matching facilities in DE
Completed matching and deduping facilities, matched 14 of 16
Matching facilities in DC
Completed matching and deduping facilities, matched 11 of 13
Matching facilities in FL
Completed [250] of 273 facilities, prelim matched 236
Completed matching and deduping facilities, matched 246 of 273
Matching fa

### Combine into one dataset

Some field names changed in the process of geocoding so we need to import the original datasets and join them using the newly created matching key to ensure that column names are consistent

In [8]:
hcris_original = gpd.read_file(processed_data_path('usa_hospital_beds_hcris2018.geojson'), encoding='utf-8')
hcris_original['Provider Number'] = hcris_original['Provider Number'].astype(str)
dh_original = gpd.read_file(external_data_path('dh_facility_data.geojson'), encoding='utf-8')
dh_original['OBJECTID'] = dh_original['OBJECTID'].astype(str)

Add the `Provider Number` (HCRIS unique ID field) to the DH dataset by joining the key to it on `OBJECTID`

In [9]:
dh_with_pn = pd.merge(dh_original, HtoD_matches[['OBJECTID', 'Provider Number']], how='left', on='OBJECTID')

Then join the HCRIS dataset to the updated DH dataset on `Provider Number`

In [10]:
full_df = pd.merge(dh_with_pn, hcris_original.drop(columns='geometry'), how='left', on='Provider Number', suffixes=('', '_HCRIS'))

Select the necessary columns by name

In [11]:
cols = ['OBJECTID', 'Provider Number', 'HOSPITAL_N', 'HOSPITAL_T', 'HQ_ADDRESS',
        'HQ_ADDRE_1', 'HQ_CITY', 'HQ_STATE', 'HQ_ZIP_COD', 'COUNTY_NAM',
        'STATE_NAME', 'STATE_FIPS', 'CNTY_FIPS', 'FIPS', 'NUM_LICENS',
        'NUM_STAFFE', 'NUM_ICU_BE', 'BED_UTILIZ', 'Potential_', 'FYB', 'FYE',
        'STATUS', 'CTRL_TYPE', 'HOSP10_Name', 'Street_Addr', 'PO_Box', 'City',
        'State', 'Zip_Code', 'County', 'Hospital Adult and Peds Staffed Beds',
        'Hospital Adult and Peds Bed Days Available',
        'Hospital Adult and Peds Inpatient Days',
        'Intensive Care Unit Staffed Beds',
        'Intensive Care Unit Bed Days Available',
        'Intensive Care Unit Inpatient Days', 'Coronary Care Unit Staffed Beds',
        'Coronary Care Unit Bed Days Available',
        'Coronary Care Unit Inpatient Days', 'Burn ICU Staffed Beds',
        'Burn ICU Bed Days Available', 'Burn ICU Inpatient Days',
        'Surgical ICU Staffed Beds', 'Surgical ICU Bed Days Available',
        'Surgical ICU Inpatient Days', 'Total Staffed Beds',
        'Total Bed Days Available', 'Total Inpatient Days',
        'ICU Total Staffed Beds', 'ICU Total Bed Days Available',
        'ICU Total Inpatient Days', 'ICU Occupancy Rate',
        'Total Bed Occupancy Rate', 'geometry']

full_df = full_df[cols]

Write off the merged geojson

In [12]:
full_df.to_file(processed_data_path('dh_hcris_merged_facility_data.geojson'), 
                encoding='utf-8', 
                driver='GeoJSON')